### Cargamos Dataset de accesos

In [2]:
import numpy as np
import pandas as pd
import openpyxl
df = pd.read_excel("../Datasets/accesos.xlsx")
df

,Acceso Puerta,Estado Acceso,Fecha Completa,Día,Nombre Día,Jornada,Semana,Mes,Mes Nombre,Mes Orden,...,Día Semana,Día Año,Mes Día,Pregrado,Postgrado,Especialidad,Carrera Pregrado,Programa Postgrado,Unnamed: 21,Unnamed: 22
0,ENTRADA,1,2023-12-14,14,JUEVES,1,50,12,DICIEMBRE,2025-12-12,...,4,348,1214,SI,NO,NO,(3309) INGENIERIA CIVIL INDUSTRIAL,NaN,NaN,NaN
1,ENTRADA,1,2023-12-14,14,JUEVES,1,50,12,DICIEMBRE,2025-12-12,...,4,348,1214,SI,NO,NO,(3309) INGENIERIA CIVIL INDUSTRIAL,NaN,NaN,NaN
2,ENTRADA,1,2023-12-14,14,JUEVES,1,50,12,DICIEMBRE,2025-12-12,...,4,348,1214,SI,NO,NO,(3506) ADMINISTRACIÓN PÚBLICA Y CIENCIA POLÍTICA,NaN,NaN,NaN
3,ENTRADA,1,2023-12-15,15,VIERNES,1,50,12,DICIEMBRE,2025-12-12,...,5,349,1215,NO,NO,NO,(3502) LICENCIATURA EN CIENCIAS JURIDICAS Y SO...,NaN,NaN,NaN
4,ENTRADA,1,2023-12-15,15,VIERNES,1,50,12,DICIEMBRE,2025-12-12,...,5,349,1215,SI,NO,NO,(3439) ANTROPOLOGIA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560684,ENTRADA,1,2025-05-07,7,MIERCOLES,1,19,5,MAYO,2025-05-05,...,3,127,507,SI,NO,NO,(3312) INGENIERIA CIVIL MECANICA,NaN,NaN,NaN
560685,ENTRADA,1,2025-05-07,7,MIERCOLES,1,19,5,MAYO,2025-05-05,...,3,127,507,SI,NO,NO,(3160) BIOINGENIERIA,NaN,NaN,NaN
560686,ENTRADA,1,2025-05-07,7,MIERCOLES,1,19,5,MAYO,2025-05-05,...,3,127,507,SI,NO,NO,(3445) SOCIOLOGIA,NaN,NaN,NaN
560687,ENTRADA,1,2025-05-07,7,MIERCOLES,1,19,5,MAYO,2025-05-05,...,3,127,507,SI,NO,NO,(3327) INGENIERIA CIVIL BIOMEDICA,NaN,NaN,NaN


### Limpiamos columnas innecesarias y agrupamos los accesos

In [ ]:
with open('../Datasets/dias_vacaciones.txt', 'r') as f:
    dias_vacaciones = [pd.to_datetime(line.strip(), errors='coerce') for line in f if line.strip()]

df['Fecha Completa'] = pd.to_datetime(df['Fecha Completa'])
df['Vacaciones?'] = df['Fecha Completa'].isin(dias_vacaciones).astype(int)

df['Día'] = df['Fecha Completa'].dt.day
df['Mes'] = df['Fecha Completa'].dt.month
df['Año'] = df['Fecha Completa'].dt.year
df['Fecha'] = df['Fecha Completa'].dt.date

# Semestre: 0 = ene-feb (vacaciones), 1 = mar-jul, 2 = ago-dic
conditions = [
    df['Mes'].isin([1, 2]),
    df['Mes'].between(3, 7),
    df['Mes'].between(8, 12)
]
choices = [0, 1, 2]
df['Semestre'] = np.select(conditions, choices).astype(int)

df_limpio = df.groupby(['Fecha', 'Día', 'Mes', 'Año', 'Semestre']).agg({
    'Fecha Completa': 'count',
    'Vacaciones?': 'max'
}).rename(columns={'Fecha Completa': 'Accesos'}).reset_index()

df_limpio

,Fecha,Día,Mes,Año,Semestre,Accesos,Vacaciones?
0,2023-12-14,14,12,2023,2,953,0
1,2023-12-15,15,12,2023,2,549,0
2,2023-12-16,16,12,2023,2,71,0
3,2023-12-18,18,12,2023,2,739,0
4,2023-12-19,19,12,2023,2,383,0
...,...,...,...,...,...,...,...
329,2025-04-29,29,4,2025,1,2568,0
330,2025-04-30,30,4,2025,1,1705,0
331,2025-05-05,5,5,2025,1,2750,0
332,2025-05-06,6,5,2025,1,1866,0


### Rellenamos fechas faltantes

In [ ]:
df_limpio['Fecha'] = pd.to_datetime(df_limpio['Fecha'])

fecha_inicio = df_limpio['Fecha'].min()
fecha_fin = df_limpio['Fecha'].max()

rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')
df_completo = pd.DataFrame({'Fecha': rango_fechas})

df_limpio_completo = df_completo.merge(
    df_limpio,
    on='Fecha',
    how='left'
)

df_limpio_completo['Accesos'] = df_limpio_completo['Accesos'].fillna(0).astype(int)

df_limpio_completo['Día'] = df_limpio_completo['Día'].fillna(df_limpio_completo['Fecha'].dt.day).astype(int)
df_limpio_completo['Mes'] = df_limpio_completo['Mes'].fillna(df_limpio_completo['Fecha'].dt.month).astype(int)
df_limpio_completo['Año'] = df_limpio_completo['Año'].fillna(df_limpio_completo['Fecha'].dt.year).astype(int)

conditions_fill= [
    df_limpio_completo['Mes'].isin([1, 2]),
    df_limpio_completo['Mes'].between(3, 7),
    df_limpio_completo['Mes'].between(8, 12)
]
choices_fill = [0, 1, 2]
df_limpio_completo['Semestre'] = np.select(
    conditions_fill,
    choices_fill,
    default=df_limpio_completo['Semestre'].fillna(-1).astype(int) # Usamos el que ya estaba si no es NaN
)
df_limpio_completo['Semestre'] = df_limpio_completo['Semestre'].replace(-1, method='ffill').astype(int)

# Asumimos que los días sin acceso son días de vacaciones
df_limpio_completo['Vacaciones?'] = df_limpio_completo['Vacaciones?'].fillna(1).astype(int)

columnas_ordenadas = ['Fecha', 'Día', 'Mes', 'Año', 'Semestre', 'Accesos', 'Vacaciones?']
df_final = df_limpio_completo[columnas_ordenadas].sort_values(by='Fecha').reset_index(drop=True)

C:\Users\tcidm\AppData\Local\Temp\ipykernel_13484\4124557598.py:32: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df_limpio_completo['Semestre'] = df_limpio_completo['Semestre'].replace(-1, method='ffill').astype(int) # Esto es una aproximación, lo correcto es el cálculo anterior.


### Guardamos el dataset de accesos agrupados en un .csv

In [5]:
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'])
df_final.to_csv('../Datasets/accesos_biblioteca.csv', index=False)